In [87]:
import numpy as np
import pandas as pd
import nltk
import spacy

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
TokenNLP = spacy.load("en_core_web_sm")

import re
from nltk.stem import WordNetLemmatizer 
from spacy import displacy
from IPython.core.display import display, HTML
from nltk.corpus import wordnet as wn

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [88]:
path = '/content/train_2.txt'
file1 = open('/content/train_2.txt', 'r')
Lines = file1.readlines()

def clean_str(text):
    text = text.lower()
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"that's", "that is ", text)
    text = re.sub(r"there's", "there is ", text)
    text = re.sub(r"it's", "it is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()


In [89]:

def preprocess():
  sentenceList = []
  relationList = []
  lines = [line.strip() for line in open(path)]
  max_sentence_length = 0
  for idx in range(0, len(lines), 4):
    id = lines[idx].split("\t")[0]
    relation = lines[idx + 1]
    sentence = lines[idx].split("\t")[1][1:-1]
    sentence = sentence.replace('<e1>', ' _e11_ ')
    sentence = sentence.replace('</e1>', ' _e12_ ')
    sentence = sentence.replace('<e2>', ' _e21_ ')
    sentence = sentence.replace('</e2>', ' _e22_ ')
    sentence = clean_str(sentence)
    relationList.append(relation)
    sentenceList.append(sentence)
  return relationList, sentenceList

relationList, sentenceList = preprocess()
print(sentenceList)
# print(len(sentenceList))
# for i in range(len(sentenceList)):
#   print(i, " ", sentenceList[i])

['e11 thom yorke e12 of e21 radiohead e22 has included the + for many of his signature distortion sounds using a variety of guitars to achieve various tonal options']


In [90]:
def getRelationsOfSentences():
  relationMap = {}
  relationCount = 0
  relationList, sentenceList = preprocess()
  for sentence in sentenceList:
    e11Index = sentence.index('e11') + 3
    e12Index = sentence.index('e12')
    firstEntity = sentence[e11Index : e12Index]

    e21Index = sentence.index('e21') + 3
    e22Index = sentence.index('e22') 
    secondEntity = sentence[e21Index : e22Index]
    #print(firstEntity, " ", secondEntity, " ", relationList[relationCount])
    relationMap[relationCount] = [firstEntity, secondEntity, relationList[relationCount]]
    relationCount += 1
  return relationMap
  

relationMap1 = getRelationsOfSentences()
print(relationMap1)

{0: [' thom yorke ', ' radiohead ', 'per:employee_of(e1,e2)']}


In [91]:

def tokenize(sentenceList):
  sentenceCount = 0
  tokenMap = {}
  for sentence in sentenceList:
    #print(sentenceCount, " ", sentence)
    sentence = sentence.replace('e11', '').replace('e12','').replace('e21','').replace('e22','')
    tokens = nltk.word_tokenize(sentence)
    tokenMap[sentenceCount] = tokens
    sentenceCount += 1
  return tokenMap
  
relationList, sentenceList = preprocess()
#print(len(sentenceList))
tokenMap = tokenize(sentenceList)
print(tokenMap)

{0: ['thom', 'yorke', 'of', 'radiohead', 'has', 'included', 'the', '+', 'for', 'many', 'of', 'his', 'signature', 'distortion', 'sounds', 'using', 'a', 'variety', 'of', 'guitars', 'to', 'achieve', 'various', 'tonal', 'options']}


In [92]:
lemmatizer = WordNetLemmatizer() 
def lemmatize(tokenMap):
  lemmaMap = {}
  lemmaCount = 0
  for tokenList in tokenMap.values():
    lemmaList = []
    #print(" tokenList ", tokenList , "/n")
    for token in tokenList:
      lemmaList.append(lemmatizer.lemmatize(token))
    lemmaMap[lemmaCount] = lemmaList
    lemmaCount += 1
  
  return lemmaMap

relationList, sentenceList = preprocess()
#print(len(sentenceList))
tokenMap = tokenize(sentenceList)
lemmaMap = lemmatize(tokenMap)
print(lemmaMap)

{0: ['thom', 'yorke', 'of', 'radiohead', 'ha', 'included', 'the', '+', 'for', 'many', 'of', 'his', 'signature', 'distortion', 'sound', 'using', 'a', 'variety', 'of', 'guitar', 'to', 'achieve', 'various', 'tonal', 'option']}


In [93]:

def posTags(tokenMap):
  posMap = {}
  posCount = 0
  for tokenList in tokenMap.values():
    posTags= nltk.pos_tag(tokenList)
    #print("posTags ", posTags)
    posMap[posCount] = posTags
    posCount += 1
  
  return posMap

relationList, sentenceList = preprocess()
tokenMap = tokenize(sentenceList)
posMap = posTags(tokenMap)
print(posMap)

{0: [('thom', 'JJ'), ('yorke', 'NN'), ('of', 'IN'), ('radiohead', 'NN'), ('has', 'VBZ'), ('included', 'VBN'), ('the', 'DT'), ('+', 'NNP'), ('for', 'IN'), ('many', 'JJ'), ('of', 'IN'), ('his', 'PRP$'), ('signature', 'NN'), ('distortion', 'NN'), ('sounds', 'VBZ'), ('using', 'VBG'), ('a', 'DT'), ('variety', 'NN'), ('of', 'IN'), ('guitars', 'NNS'), ('to', 'TO'), ('achieve', 'VB'), ('various', 'JJ'), ('tonal', 'JJ'), ('options', 'NNS')]}


In [115]:
lemmatizer = WordNetLemmatizer() 
def getHyponymsAndHypernyms(tokenMap):
  for sentence in sentenceList:
    sentence = sentence.replace('e11', " ").replace('e12'," ").replace('e21'," ").replace('e22'," ")
    tokendoc = TokenNLP(sentence)
    print("sentence is ", sentence)
    for token in tokendoc:
      print(" token is ", token)
      for ss in wn.synsets(token.lemma_):
        
        print ("hypernyms: "+str(ss.hypernyms())+" hyponyms: " + str(ss.hyponyms())+" holonyms: " + str(ss.member_holonyms())+" meronyms: "+ str(ss.part_meronyms()))
          
    
    
  

relationList, sentenceList = preprocess()
#print(len(sentenceList))
#tokenMap = tokenize(sentenceList)
getHyponymsAndHypernyms(sentenceList)


sentence is    thom yorke   of   radiohead   has included the + for many of his signature distortion sounds using a variety of guitars to achieve various tonal options
 token is    
 token is  thom
 token is  yorke
 token is    
 token is  of
 token is    
 token is  radiohead
 token is    
 token is  has
hypernyms: [Synset('person.n.01')] hyponyms: [Synset('affluent.n.01'), Synset('billionaire.n.01'), Synset('croesus.n.02'), Synset('fat_cat.n.01'), Synset('man_of_means.n.01'), Synset('millionaire.n.01'), Synset('millionairess.n.01'), Synset('multi-billionaire.n.01'), Synset('plutocrat.n.01')] holonyms: [] meronyms: []
hypernyms: [] hyponyms: [Synset('bear.v.11'), Synset('carry.v.21'), Synset('keep.v.03'), Synset('keep.v.07'), Synset('keep.v.19'), Synset('keep.v.20'), Synset('monopolize.v.02'), Synset('stock.v.01'), Synset('sustain.v.04'), Synset('wield.v.01')] holonyms: [] meronyms: []
hypernyms: [] hyponyms: [Synset('abound.v.02'), Synset('bear.v.01'), Synset('brim.v.01'), Synset('br

In [95]:
def NERTags(tokenMap):
  NERMap = {}
  NERCount = 0
  for firstEntity, secondEntity, relationList in getRelationsOfSentences().values():
    docFirst = nlp(firstEntity)
    docSecond = nlp(secondEntity)
    NERList = []
    #print("posTags ", posTags)
    for ent in docFirst.ents:
      NERList.append((ent.text, ent.label_))
    for ent in docSecond.ents:
      NERList.append((ent.text, ent.label_))
    NERMap[NERCount] = NERList
    NERCount += 1
  
  return NERMap


NERMap = NERTags(getRelationsOfSentences())
print(NERMap)

{0: []}


In [98]:

def dependencyTree(sentenceList):
  
  dependecyMap = {}
  dependecyCount = 0
  for sentence in sentenceList:
    sentence = sentence.replace('e11', " ").replace('e12'," ").replace('e21'," ").replace('e22'," ")
    depedenlist = []
    tokendoc = TokenNLP(sentence)
    for token in tokendoc:
      depedenlist.append((token,token.dep_))
    dependecyMap[dependecyCount] = depedenlist
    dependecyCount += 1
    html = displacy.render(tokendoc, style="dep")
    # Uncomment below 2 lines
    # print(" Sentence is ", sentence)
    # display(HTML(html))
  print(dependecyMap)

relationList, sentenceList = preprocess()
dependencyTree(sentenceList)

{0: [(  , ''), (thom, 'compound'), (yorke, 'nsubj'), (  , ''), (of, 'prep'), (  , ''), (radiohead, 'pobj'), (  , ''), (has, 'aux'), (included, 'ROOT'), (the, 'det'), (+, 'dobj'), (for, 'mark'), (many, 'nsubj'), (of, 'prep'), (his, 'poss'), (signature, 'compound'), (distortion, 'pobj'), (sounds, 'conj'), (using, 'xcomp'), (a, 'det'), (variety, 'dobj'), (of, 'prep'), (guitars, 'pobj'), (to, 'aux'), (achieve, 'xcomp'), (various, 'amod'), (tonal, 'amod'), (options, 'dobj')]}
